# Entanglement Distillation -- DEJMPS Protocol


<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## Overview

Before reading this tutorial, we highly recommend you to read the [BBPSSW protocol](./EntanglementDistillation_BBPSSW_EN.ipynb) first if you are not familiar with entanglement distillation. The DEJMPS protocol, introduced by Deutsch et al. [1], is similar to the BBPSSW protocol. The main difference between these two protocols is the state for distillation: the DEJMPS protocol can distill Bell-diagonal states, while the BBPSSW protocol could distill isotropic states. In entanglement distillation, the main purpose is to generate a **maximally entangled state** $|\Phi^+\rangle$ from many copies of imperfect entangled states, using only LOCC operations. Recall the four Bell states,

$$ 
\begin{align*}
|\Phi^{\pm}\rangle_{AB} &= \frac{1}{\sqrt{2}}(|0\rangle_A\otimes|0\rangle_B \pm |1\rangle_A\otimes|1\rangle_B), \\
|\Psi^{\pm}\rangle_{AB} &= \frac{1}{\sqrt{2}}(|0\rangle_A\otimes|1\rangle_B \pm |1\rangle_A\otimes|0\rangle_B). 
\tag{1}
\end{align*}
$$

where $A$ and $B$ represent the bi-party Alice and Bob. The Bell-diagonal state, by definition, is diagonal in the Bell basis that can be expressed as

$$
\rho_{\text{diag}} = p_1 | \Phi^+\rangle \langle \Phi^+ | + p_2 | \Psi^+\rangle \langle \Psi^+ | + 
p_3 | \Phi^-\rangle \langle \Phi^- |  + p_4 | \Psi^-\rangle \langle \Psi^- |.
\tag{2}
$$

with $p_1 > p_2 \geq p_3 \geq p_4$ and $p_1 + p_2+ p_3+ p_4 = 1$. Then the entanglement quantification of a Bell-diagonal state can be described as:

* State fidelity $F = \langle \Phi^+|\rho_{\text{diag}}|\Phi^+\rangle = p_1$
* Negativity $\mathcal{N}(\rho_{\text{diag}}) = p_1 - 1/2$

**Note:** The Bell-diagonal state is distillable when $p_1 > 1/2$.

## DEJMPS protocol

Suppose that two parties, namely Alice($A$) and Bob($B$), possess two copies of entangled qubit: $\{ A_0, B_0 \}, \{ A_1, B_1 \}$. If these two pairs are all in the same Bell-diagonal state $\rho_{\text{diag}}$, with $p_1 > 0.5$. We can apply the following workflow to purify the input states and leads to an output state has fidelity closer to $|\Phi^+\rangle$:
1. Alice and Bob firstly choose the qubit pair **they want to keep as the memory qubit pair after distillation**. Here, they choose $A_0$ and $B_0$. 
2. Alice performs $R_x(\pi/2)$ gates on both qubits, and Bob performs $R_x(-\pi/2)$ gates on both qubits.
3. Then, Alice and Bob both apply a CNOT gate on their qubits. Here, they choose $A_0,B_0$ as the control qubits and $A_1,B_1$ as the target qubits.
4. Two remote parties measure the target qubits and use a classical communication channel to exchange their measurement results $m_{A_1}, m_{B_1}$.
5. If the measurement results of Alice and Bob are the same (00 or 11), the distillation is successful, and the qubit pair $A_0, B_0$ is stored as state $\rho_{out}$; If the measurement results are different (01 or 10), they claim the distillation failed and the qubit pair $A_0, B_0$ will be discarded.

<center><img src="figures/distillation-fig-DEJMPS.jpg" height="250" width="300"></center>
<div style="text-align:center">Figure 1: Schematic diagram of the DEJMPS protocol </div>

After the distillation, the final state $\rho_{out}$ of entangled pair $A_0, B_0$ will have higher fidelity than the initial state $\rho$. The fidelity of the final state $F_{out}$ is

$$
F_{out} = \frac{p_1^2 + p_4^2}{(p_1 + p_4)^2 + (p_2 + p_3)^2},
\tag{3}
$$

Similar to the BBPSSW protocol, the DEJMPS protocol is probabilistic, with the probability of a successful distillation being 

$$ 
p_{succ} = (p_1 + p_4)^2 + (p_2 + p_3)^2.
\tag{4}
$$

## Simulation with Paddle Quantum
First, we need to import relevant packages

In [1]:
import numpy as np
import paddle.fluid as fluid
from paddle_quantum.locc import LoccNet
from paddle.complex import matmul, trace
from paddle_quantum.state import bell_state, isotropic_state, bell_diagonal_state
from paddle_quantum.utils import negativity, logarithmic_negativity, is_ppt

First, let us see the theoretical result of applying the **DEJMPS protocol** to the state

$$
\rho = p_1 | \Phi^+\rangle \langle \Phi^+ | + \frac{1-p_1}{2} | \Psi^+\rangle \langle \Psi^+ |+ 
\frac{1-p_1}{3}| \Phi^-\rangle \langle \Phi^- |  + \frac{1-p_1}{6} | \Psi^-\rangle \langle \Psi^- |.
\tag{5}
$$

Suppose we take $p_1 = 0.7$, then the theoretical improvement of fidelity can be calculated by the following block:

In [2]:
def DEJMPS_metrics(*p):
    """
    Returns output fidelity and probability of success of the DEJMPS protocol.
    """
    F_in = p[0]
    p_succ = (p[0] + p[3]) ** 2 + (p[1] + p[2]) ** 2
    F_out = (p[0] ** 2 + p[3] ** 2)/p_succ
 
    return F_in, F_out, p_succ

p = 0.7
F_in, F_out, p_succ = DEJMPS_metrics(p, (1-p)/2, (1-p)/3, (1-p)/6)
print("The input fidelity is:", F_in)
print("The output fidelity is:", F_out)
print("With a probability of success:", p_succ)
print("The input state satisfies the PPT condition and hence not distillable?", 
      is_ppt(bell_diagonal_state(p, (1-p)/2, (1-p)/3, (1-p)/6)))

The input fidelity is: 0.7
The output fidelity is: 0.7879999999999999
With a probability of success: 0.625
The input state satisfies the PPT condition and hence not distillable? False


Then we can simulate the DEJMPS protocol and check if the results match with the theory.

In [4]:
class LOCC(LoccNet):
    def __init__(self):
        super(LOCC, self).__init__()
        
        # Add the first party Alice 
        # The first parameter 2 stands for how many qubits A holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(2, party_name='Alice')
        
        # Add the second party Bob
        # The first parameter 2 stands for how many qubits A holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(2, party_name='Bob')
        
        # Define the input quantum states rho_in
        _state = fluid.dygraph.to_variable(bell_diagonal_state(p, (1-p)/2, (1-p)/3, (1-p)/6))
        
        # ('Alice', 0) means Alice's first qubit A0
        # ('Bob', 0) means Bob's first qubit B0
        self.set_init_status(_state, [('Alice', 0), ('Bob', 0)]) 
        
        # ('Alice', 1) means Alice's second qubit A1
        # ('Bob', 1) means Bob's second qubit B1
        self.set_init_status(_state, [('Alice', 1), ('Bob', 1)])  
        
        # Set the angles of the Rx gates
        self.theta1 = fluid.dygraph.to_variable(np.array([np.pi/2, np.pi/2], dtype='float64'))
        self.theta2 = fluid.dygraph.to_variable(np.array([-np.pi/2, -np.pi/2], dtype='float64'))
        
    def DEJMPS(self):
        status = self.init_status
        
        # Create Alice's local operations 
        cir1 = self.create_ansatz('Alice')
        cir1.rx(self.theta1[0], 0)
        cir1.rx(self.theta1[1], 1)
        cir1.cnot([0, 1])

        # Create Bob's local operations 
        cir2 = self.create_ansatz('Bob')
        cir2.rx(self.theta2[0], 0)
        cir2.rx(self.theta2[1], 1)
        cir2.cnot([0, 1])
    
        # Run circuit
        status = cir1.run(status)
        status_mid = cir2.run(status)
        
        # ('Alice', 1) means measuring Alice's second qubit A1
        # ('Bob', 1) means measuring Bob's second qubit B1
        # ['00','11'] specifies the success condition for distillation
        # Means Alice and Bob both measure '00' or '11'
        status_mid = self.measure(status_mid, [('Alice', 1), ('Bob', 1)], ["00", "11"])
        
        # Trace out the measured qubits A1&B1
        # Leaving only Alice’s first qubit and Bob’s first qubit A0&B0 as the memory register
        status_fin = self.partial_state(status_mid, [('Alice', 0), ('Bob', 0)])
        
        return status_fin
    

# Turn on dynamic computational graph
with fluid.dygraph.guard():
    
    # Run DEJMPS protocol
    status_fin = LOCC().DEJMPS()
    
    # Calculate fidelity
    target_state = fluid.dygraph.to_variable(bell_state(2))
    fidelity = 0
    for status in status_fin:
        fidelity += trace(matmul(target_state, status.state)).real
    fidelity /= len(status_fin)
    
    # Calculate success rate
    suc_rate = sum([status.prob for status in status_fin])
    
    # Output simulation results
    print(f"The fidelity of the input quantum state is: {p:.5f}")
    print(f"The fidelity of the purified quantum state is: {fidelity.numpy()[0]:.5f}")
    print(f"The probability of successful purification is: {suc_rate.numpy()[0]:#.3%}")
    
    # Print the output state
    rho_out = status_fin[0].state.numpy()
    print("========================================================")
    print(f"The output state is:\n {np.around(rho_out,4)}")
    print(f"The initial negativity is: {negativity(bell_diagonal_state(p, (1-p)/2, (1-p)/3, (1-p)/6))}")
    print(f"The final negativity is: {negativity(rho_out)}")

The fidelity of the input quantum state is: 0.70000
The fidelity of the purified quantum state is: 0.78800
The probability of successful purification is: 62.500%
The output state is:
 [[0.45 -0.j 0.   +0.j 0.   -0.j 0.338-0.j]
 [0.   -0.j 0.05 +0.j 0.002-0.j 0.   +0.j]
 [0.   -0.j 0.002+0.j 0.05 -0.j 0.   -0.j]
 [0.338+0.j 0.   +0.j 0.   +0.j 0.45 -0.j]]
The initial negativity is: 0.19999999999999993
The final negativity is: 0.2880000000000001


One can observe that the simulation result is in exact accordance with the theoretical values.

## Conclusion

The DEJMPS protocol can effectively extract one entangled pair with higher fidelity from two noisy pairs. And compared to the BBPSSW protocol [2], it can be applied to Bell-diagonal states instead of isotropic states. Note that isotropic state is a special case of Bell-diagonal state. So in this sense, the DEJMPS protocol is more general than the BBPSSW protocol. However, it also shares the same disadvantages of the BBPSSW protocol including the limited type of input states and poor scalability. 

Next, We suggest interested readers to check the tutorial on [how to design a new distillation protocol with LOCCNet](./EntanglementDistillation_LOCCNet_EN.ipynb).

---

## References

[1] Deutsch, David, et al. "Quantum privacy amplification and the security of quantum cryptography over noisy channels." [Physical Review Letters 77.13 (1996): 2818.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.2818)

[2] Bennett, Charles H., et al. "Purification of noisy entanglement and faithful teleportation via noisy channels." [Physical Review Letters 76.5 (1996): 722.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.76.722)
